# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [95]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('/Users/xp/production/05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
adult_dt.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [96]:
print(adult_dt.columns.tolist())

['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']


In [97]:
from sklearn.model_selection import train_test_split
X= adult_dt[['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',  
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country']]
y =adult_dt['income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)* Random state is a seed for the random number generator. Random states ensure that experiments are consistent and reliable. It gives us the same radom selection of value even after running code multiple times ensuring the reproducibility.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [98]:
numerical_features=['age','education-num','capital-gain','capital-loss','hours-per-week']
categorical_features=['workclass','education','marital-status','occupation','relationship','race','sex','native-country']

In [99]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 
         Pipeline(steps=[
             ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),
             ('scaler', RobustScaler())
         ]), 
         numerical_features),
        
        ('cat', 
         Pipeline(steps=[
             ('simple_imputer', SimpleImputer(strategy='most_frequent')),
             ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first'))
         ]), 
         categorical_features)
    ],
    remainder='drop'  # Drop any other features not specified
)

model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(criterion='entropy', max_depth=3))
])



## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [113]:
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # Step for preprocessing
    ('classifier', RandomForestClassifier(criterion='entropy', max_depth=3))  # Step for classifier
])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [116]:
from sklearn.model_selection import cross_validate

scoring = ['neg_log_loss', 'roc_auc', 'accuracy', 'balanced_accuracy']

cv_results = cross_validate(
    pipeline, 
    X,  # Features
    y,  # Labels
    scoring=scoring, 
    cv=5, 
    return_train_score=True
)



/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [115]:
results_df = pd.DataFrame(cv_results)

# Sort by negative log loss (neg_log_loss is returned as negative, so we sort ascending)
results_df = results_df.sort_values(by='test_neg_log_loss')

# Display the sorted DataFrame
print(results_df)

   fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
4  0.359117    0.059369          -0.424220           -0.425677      0.897537   
1  0.375707    0.058960          -0.422282           -0.419057      0.893479   
2  0.362269    0.058259          -0.421807           -0.421007      0.896857   
0  0.386290    0.059481          -0.419089           -0.417887      0.894202   
3  0.369850    0.061012          -0.417575           -0.418705      0.899161   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
4       0.893883       0.776106        0.773811                0.535077   
1       0.901470       0.786087        0.786710                0.555804   
2       0.897980       0.781480        0.782410                0.546237   
0       0.898469       0.784738        0.783822                0.553436   
3       0.895172       0.791155        0.790088                0.566762   

   train_balanced_accuracy  
4                 0.530532  
1         

Calculate the mean of each metric. 

In [105]:
# Calculate the mean of each metric
mean_metrics = results_df.mean()

# Display the mean metrics
print(mean_metrics)



fit_time                   0.374090
score_time                 0.059689
test_neg_log_loss         -0.419024
train_neg_log_loss        -0.418458
test_roc_auc               0.896397
train_roc_auc              0.897354
test_accuracy              0.786309
train_accuracy             0.786071
test_balanced_accuracy     0.556478
train_balanced_accuracy    0.555987
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [117]:

model_pipeline.fit(X_train, y_train)

# Get prediction probabilities for the test set
Y_pred_proba = model_pipeline.predict_proba(X_test)

# Calculate metrics
roc_auc = roc_auc_score(y_test, Y_pred_proba[:, 1])
neg_log_loss = log_loss(y_test, Y_pred_proba)
accuracy = accuracy_score(y_test, model_pipeline.predict(X_test))
balanced_accuracy = balanced_accuracy_score(y_test, model_pipeline.predict(X_test))

# Store results in a dictionary
performance_metrics = {
    'Negative Log Loss': neg_log_loss,
    'ROC AUC': roc_auc,
    'Accuracy': accuracy,
    'Balanced Accuracy': balanced_accuracy
}

# Display results
print(performance_metrics)


{'Negative Log Loss': 0.42213786989755947, 'ROC AUC': 0.8991731141675753, 'Accuracy': 0.7884123247005835, 'Balanced Accuracy': 0.5533707865168539}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)Many evaluation metrics in scikit-learn, like accuracy, precision, recall, ROC AUC, and log loss, are designed to work directly with binary or numeric target variables. Recoding the target variable up front allows for straightforward calculation of these metrics without needing further transformation.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.